In [2]:
import pandas as pd
from datetime import datetime
import ta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data = pd.read_csv("SBIN_EQ.csv")
data.head()

,date,open,high,low,close,volume
0,2017-07-03 09:15:00,274.20,275.00,274.2,274.65,112783
1,2017-07-03 09:16:00,274.55,274.70,274.2,274.30,67471
2,2017-07-03 09:17:00,274.35,274.45,274.1,274.20,105108
3,2017-07-03 09:18:00,274.20,274.20,273.8,273.90,51453
4,2017-07-03 09:19:00,273.85,274.00,273.6,273.65,45098


In [5]:
data["Date"] = data["date"].str.split(" ").str[0]
data["Time"] = data["date"].str.split(" ").str[1].str.split("+").str[0]
data["Date_Time"] = data['Date'] + " " + data["Time"]
data.drop("Date", axis = 1, inplace = True)
data.head()



,date,open,high,low,close,volume,Time,Date_Time
0,2017-07-03 09:15:00,274.20,275.00,274.2,274.65,112783,09:15:00,2017-07-03 09:15:00
1,2017-07-03 09:16:00,274.55,274.70,274.2,274.30,67471,09:16:00,2017-07-03 09:16:00
2,2017-07-03 09:17:00,274.35,274.45,274.1,274.20,105108,09:17:00,2017-07-03 09:17:00
3,2017-07-03 09:18:00,274.20,274.20,273.8,273.90,51453,09:18:00,2017-07-03 09:18:00
4,2017-07-03 09:19:00,273.85,274.00,273.6,273.65,45098,09:19:00,2017-07-03 09:19:00


In [6]:
data["Date_Time"] = pd.to_datetime(data["Date_Time"], format = "%Y-%m-%d %H:%M:%S")
data.index = data['Date_Time']
data.drop(["date"], axis = 1, inplace = True)
data["Next Date_Time"] = data["Date_Time"].shift(-1)
data.head()

,open,high,low,close,volume,Time,Date_Time,Next Date_Time
Date_Time,,,,,,,,
2017-07-03 09:15:00,274.20,275.00,274.2,274.65,112783,09:15:00,2017-07-03 09:15:00,2017-07-03 09:16:00
2017-07-03 09:16:00,274.55,274.70,274.2,274.30,67471,09:16:00,2017-07-03 09:16:00,2017-07-03 09:17:00
2017-07-03 09:17:00,274.35,274.45,274.1,274.20,105108,09:17:00,2017-07-03 09:17:00,2017-07-03 09:18:00
2017-07-03 09:18:00,274.20,274.20,273.8,273.90,51453,09:18:00,2017-07-03 09:18:00,2017-07-03 09:19:00
2017-07-03 09:19:00,273.85,274.00,273.6,273.65,45098,09:19:00,2017-07-03 09:19:00,2017-07-03 09:20:00


In [9]:
def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in data.index:
        step += 1
        if step == 1:
            o = data.loc[i, "open"]
            t = i
            h = data.loc[i, "high"]
            l = data.loc[i, "low"]
        else:
            if data.loc[i, "high"] > h:
                h = data.loc[i, "high"]
            if data.loc[i, "low"] < l:
                l = data.loc[i, "low"]
            if step == tf or i.day != data.loc[i, "Next_Date_Time"].day:
                step = 0
                c = data.loc[i, "close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
    
    return pd.DataFrame(new_data)


In [10]:
def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in range(len(data)):
        step += 1
        if step == 1:
            o = data.iloc[i]["open"]
            t = data.index[i]
            h = data.iloc[i]["high"]
            l = data.iloc[i]["low"]
        else:
            if data.iloc[i]["high"] > h:
                h = data.iloc[i]["high"]
            if data.iloc[i]["low"] < l:
                l = data.iloc[i]["low"]
            if step == tf or (i+1) == len(data) or data.index[i+1].day != data.index[i].day:
                step = 0
                c = data.iloc[i]["close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
                if (i+1) < len(data):
                    t = data.index[i+1]
                    o = data.iloc[i+1]["open"]
                    h = data.iloc[i+1]["high"]
                    l = data.iloc[i+1]["low"]
    
    return pd.DataFrame(new_data)


In [12]:
data45 = convert_timeframe(data, 45)
data45_original = data45.copy()
data45.head()



,Date_Time,Open,High,Low,Close
0,2017-07-03 09:15:00,274.20,275.00,272.55,274.45
1,2017-07-03 10:00:00,274.50,274.85,273.70,274.35
2,2017-07-03 10:45:00,274.35,275.30,274.25,274.30
3,2017-07-03 11:30:00,274.35,275.25,274.25,274.90
4,2017-07-03 12:15:00,274.90,275.45,274.70,275.05


In [13]:
data45.describe()

,Date_Time,Open,High,Low,Close
count,14947,14947.000000,14947.000000,14947.000000,14947.000000
mean,2020-11-13 10:06:35.734260736,390.283656,391.701736,388.786971,390.244036
min,2017-07-03 09:15:00,150.650000,151.200000,149.450000,150.600000
25%,2019-03-07 14:07:30,276.350000,277.500000,275.325000,276.350000
50%,2020-11-12 10:00:00,336.400000,337.700000,335.050000,336.450000
75%,2022-07-19 11:52:30,519.125000,521.000000,517.200000,519.075000
max,2024-03-21 15:15:00,791.500000,793.400000,788.250000,791.450000
std,NaN,143.413968,143.702184,143.128173,143.419958


5EMA on 45 min Short Positions only

In [14]:
status = None
alert_candle = None
position = None
short_at = None
stop_loss = None

data45 = data45_original.copy()
data45["5EMA"] = ta.trend.EMAIndicator(data45["Close"], 5).ema_indicator()
day = None

for i in data45.index:
 
 if position == None:

    if alert_candle == None: # Previous Candle is not an alert candle

        if data45.loc[i, "5EMA"] < data45.loc[i, "Low"]: # Alert Candle
            alert_candle = i
            data45.loc[i, "ALERT"] = "ALERT"
    else: # Previous Candle is an alert candle -> Try to take postion


        if data45.loc[i, "Low"] < data45.loc[alert_candle, "Low"]:
            position = "SHORT"
            data45.loc[i, "POSITION"] = "ENTRY"

            if data45.loc[i, "Date_Time"].day != day:
                data45.loc[i, "Days fund required"] = 1
                day = data45.loc[i, "Date_Time"].day
            short_at = data45.loc[alert_candle, "Low"]

            stop_loss = data45.loc[alert_candle, "High"]
            original_sl = stop_loss
            target = short_at - 3*(stop_loss - short_at)

            data45.loc[i, "TARGET"] = target
            data45.loc[i, "SL"] = stop_loss
            data45.loc[i, "ENTRY_AT"] = short_at
            alert_candle = None

        elif data45.loc[i, "5EMA"] < data45.loc[i, "Low"]: # Check for once again aler
            alert_candle = i
            data45.loc[i, "ALERT"] = "ALERT"
        else:
            alert_candle = None

 else: # I already have a short position
    data45.loc[i, "POSITION"] = "HOLD"
    data45.loc[i, "SL"] = stop_loss
    data45.loc[i, "TARGET"] = target
    data45.loc[i, "ENTRY_AT"] = short_at

    if data45.loc[i, "Low"] <= target:
        data45.loc[i, "POSITION"] = "TRAIL"

        temp = target - (original_sl - short_at)
        stop_loss = target
        target = temp
        data45.loc[i, "SL"] = stop_loss
        data45.loc[i, "TARGET"] = target

    elif data45.loc[i, "High"] >= stop_loss:
            data45.loc[i, "POSITION"] = "EXIT"
            position = None
            profit = short_at - stop_loss
            data45.loc[i, "PROFIT"] = profit
            data45.loc[i, "EXIT_AT"] = stop_loss

In [15]:
data45.iloc[4222:4280] # Lets check any two random interval


,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
4222,2019-05-29 13:45:00,348.55,349.00,347.50,348.15,349.716767,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4223,2019-05-29 14:30:00,348.20,349.50,347.50,348.95,349.461178,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4224,2019-05-29 15:15:00,348.85,349.00,347.95,348.20,349.040785,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4225,2019-05-30 09:15:00,349.75,350.85,347.50,349.90,349.327190,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4226,2019-05-30 10:00:00,349.90,352.20,349.60,350.60,349.751460,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4227,2019-05-30 10:45:00,350.65,351.10,349.65,350.35,349.950973,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4228,2019-05-30 11:30:00,350.40,351.00,349.35,350.65,350.183982,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4229,2019-05-30 12:15:00,350.65,351.50,350.45,350.95,350.439322,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4230,2019-05-30 13:00:00,350.95,352.50,350.05,352.00,350.959548,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN
4231,2019-05-30 13:45:00,352.10,352.10,350.80,351.50,351.139698,nan,HOLD,NaN,337.15,363.35,356.8,NaN,NaN


In [16]:
data45.index = data45["Date_Time"]
data45["PROFIT"].resample("A").sum()


Date_Time
2017-12-31     53.40
2018-12-31    162.60
2019-12-31    107.15
2020-12-31    219.20
2021-12-31    111.35
2022-12-31    257.25
2023-12-31    175.70
2024-12-31     40.30
Freq: A-DEC, Name: PROFIT, dtype: float64

In [17]:
data45.head()

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
Date_Time,,,,,,,,,,,,,,
2017-07-03 09:15:00,2017-07-03 09:15:00,274.20,275.00,272.55,274.45,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-03 10:00:00,2017-07-03 10:00:00,274.50,274.85,273.70,274.35,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-03 10:45:00,2017-07-03 10:45:00,274.35,275.30,274.25,274.30,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-03 11:30:00,2017-07-03 11:30:00,274.35,275.25,274.25,274.90,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-03 12:15:00,2017-07-03 12:15:00,274.90,275.45,274.70,275.05,274.717901,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#data45.to_excel("5EMA_Sell_SBIN.xlsx")

5EMA 45min Buy Only

In [21]:
data45B = convert_timeframe(data, 45)
data45B_original = data45B.copy()
data45B.head()

,Date_Time,Open,High,Low,Close
0,2017-07-03 09:15:00,274.20,275.00,272.55,274.45
1,2017-07-03 10:00:00,274.50,274.85,273.70,274.35
2,2017-07-03 10:45:00,274.35,275.30,274.25,274.30
3,2017-07-03 11:30:00,274.35,275.25,274.25,274.90
4,2017-07-03 12:15:00,274.90,275.45,274.70,275.05


In [22]:
status = None
alert_candle = None
position = None
long_at = None
stop_loss = None

data45B = data45B_original.copy()
data45B["5EMA"] = ta.trend.EMAIndicator(data45B["Close"], 5).ema_indicator()
day = None

for i in data45B.index:
 
 if position == None:

    if alert_candle == None: # Previous Candle is not an alert candle

        if data45B.loc[i, "5EMA"] > data45B.loc[i, "High"]: # Alert Candle
            alert_candle = i
            data45B.loc[i, "ALERT"] = "ALERT"

    else: # Previous Candle is an alert candle -> Try to take postion


        if data45B.loc[i, "High"] > data45B.loc[alert_candle, "High"]:
            position = "LONG"
            data45B.loc[i, "POSITION"] = "ENTRY"

            if data45B.loc[i, "Date_Time"].day != day:
                data45B.loc[i, "Days fund required"] = 1
                day = data45B.loc[i, "Date_Time"].day

            long_at = data45B.loc[alert_candle, "High"]

            stop_loss = data45B.loc[alert_candle, "Low"]
            original_sl = stop_loss
            target = long_at + 3*(long_at - stop_loss)

            data45B.loc[i, "TARGET"] = target
            data45B.loc[i, "SL"] = stop_loss
            data45B.loc[i, "ENTRY_AT"] = long_at
            alert_candle = None

        elif data45B.loc[i, "5EMA"] > data45B.loc[i, "High"]: # Check for once again alert
            alert_candle = i
            data45B.loc[i, "ALERT"] = "ALERT"
        else:
            alert_candle = None

 else: # I already have a long position
    data45B.loc[i, "POSITION"] = "HOLD"
    data45B.loc[i, "SL"] = stop_loss
    data45B.loc[i, "TARGET"] = target
    data45B.loc[i, "ENTRY_AT"] = long_at

    if data45B.loc[i, "High"] >= target:
        data45B.loc[i, "POSITION"] = "TRAIL"

        temp = target + (long_at - original_sl)
        stop_loss = target
        target = temp
        data45B.loc[i, "SL"] = stop_loss
        data45B.loc[i, "TARGET"] = target

    elif data45B.loc[i, "Low"] <= stop_loss:
            data45B.loc[i, "POSITION"] = "EXIT"
            position = None
            profit = stop_loss - long_at
            data45B.loc[i, "PROFIT"] = profit
            data45B.loc[i, "EXIT_AT"] = stop_loss


In [24]:
data45B.loc[3353:3373] # Lets check any two random interval

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
3353,2019-01-07 10:00:00,298.60,300.10,298.45,299.90,298.289536,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3354,2019-01-07 10:45:00,299.90,299.95,297.50,298.95,298.509691,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3355,2019-01-07 11:30:00,298.95,298.95,297.90,298.45,298.489794,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3356,2019-01-07 12:15:00,298.40,298.90,298.00,298.90,298.626529,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3357,2019-01-07 13:00:00,298.85,298.90,297.15,297.45,298.234353,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3358,2019-01-07 13:45:00,297.45,297.70,296.60,297.20,297.889569,ALERT,nan,NaN,NaN,NaN,NaN,NaN,NaN
3359,2019-01-07 14:30:00,297.30,297.35,295.20,296.55,297.443046,ALERT,nan,NaN,NaN,NaN,NaN,NaN,NaN
3360,2019-01-07 15:15:00,296.50,296.90,296.15,296.55,297.145364,ALERT,nan,NaN,NaN,NaN,NaN,NaN,NaN
3361,2019-01-08 09:15:00,295.80,297.65,294.75,297.20,297.163576,nan,ENTRY,1.0,299.15,296.15,296.9,NaN,NaN
3362,2019-01-08 10:00:00,297.10,298.50,297.10,297.70,297.342384,nan,HOLD,NaN,299.15,296.15,296.9,NaN,NaN


In [25]:
data45B.describe()

,Date_Time,Open,High,Low,Close,5EMA,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
count,14947,14947.000000,14947.000000,14947.000000,14947.000000,14943.000000,745.0,6139.000000,6139.000000,6139.000000,873.000000,873.000000
mean,2020-11-13 10:06:35.734260736,390.283656,391.701736,388.786971,390.244036,390.212145,1.0,393.854357,381.720777,383.173367,1.212314,380.563001
min,2017-07-03 09:15:00,150.650000,151.200000,149.450000,150.600000,151.070727,1.0,153.500000,151.500000,152.000000,-19.050000,152.400000
25%,2019-03-07 14:07:30,276.350000,277.500000,275.325000,276.350000,276.276947,1.0,280.000000,272.250000,273.000000,-2.250000,273.850000
50%,2020-11-12 10:00:00,336.400000,337.700000,335.050000,336.450000,336.260146,1.0,349.050000,332.000000,332.450000,-1.300000,321.850000
75%,2022-07-19 11:52:30,519.125000,521.000000,517.200000,519.075000,519.175446,1.0,519.350000,505.500000,508.100000,4.350000,510.550000
max,2024-03-21 15:15:00,791.500000,793.400000,788.250000,791.450000,788.872901,1.0,786.200000,770.550000,774.100000,39.600000,770.550000
std,NaN,143.413968,143.702184,143.128173,143.419958,143.349692,0.0,140.465216,138.114843,138.537283,6.344477,142.541152


In [26]:
data45B.index = data45B["Date_Time"]
data45B["PROFIT"].resample("A").sum()


Date_Time
2017-12-31     73.75
2018-12-31    167.55
2019-12-31    126.55
2020-12-31     80.00
2021-12-31    192.55
2022-12-31    213.15
2023-12-31    152.55
2024-12-31     52.25
Freq: A-DEC, Name: PROFIT, dtype: float64